<a href="https://colab.research.google.com/github/Jontpan/master-thesis/blob/master/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
import numpy as np
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def read_qaqc_split(filename):
    texts = []
    labels = []

    with open('drive/MyDrive/master-thesis/data/' + filename + '_texts.txt', 'r') as f:
        texts = f.readlines()

    with open('drive/MyDrive/master-thesis/data/' + filename + '_labels.txt', 'r') as f:
        labels = f.readlines()
    
    t = [x.strip() for x in texts]
    l = [x.strip() for x in labels]
    l2 = label_to_index(l)

    return pd.DataFrame(list(zip(t, l, l2)), columns=['text', 'verbose label', 'label'])

def label_to_index(labels):
    converter = []
    converted_labels = []

    with open('drive/MyDrive/master-thesis/data/labels.txt', 'r') as f:
        converter = f.readlines()
    
    converter = [x.strip() for x in converter]
    
    for idx, label in enumerate(labels):
        for i, e in enumerate(converter):
            if label == e:
                converted_labels.append(i)
                break
        
        else:
            print(label, idx)
    
    assert(len(converted_labels) == len(labels))

    return converted_labels

In [ ]:
train_data = read_qaqc_split('train')
test_data = read_qaqc_split('test')

train_data.sample(5)

,text,verbose label,label
1432,Hur många penny-farthings finns det på ett Tri...,NUM:count,38
4148,"Vad är en ""falsk måne""?",DESC:def,2
3542,"Varför är universum platt, om det började med ...",DESC:reason,5
4324,Vem är regissör och redaktör för filmen Big me...,HUM:ind,30
888,Vilka upptäcktsresande följde Columbus till Am...,HUM:ind,30


In [ ]:
from sklearn.model_selection import train_test_split

X = train_data.text.values
y = train_data.label.values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.2)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

print(X_train_counts.shape)

(4762, 9678)


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer()
X_train_tf = tf_transformer.fit_transform(X_train_counts)

print(X_train_tf.shape)
print(y_train.shape)

(4762, 9678)
(4762,)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tf, y_train)

In [ ]:
X_new_counts = count_vect.transform(X_val)
X_new_tf = tf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tf)

print(np.mean(predicted == y_val))



0.4265323257766583


In [ ]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None).fit(X_train_tf, y_train)

predicted = clf.predict(X_new_tf)

print(np.mean(predicted == y_val))

0.6842989084802686
